# Start

In [1]:
import importlib
import os
import pandas as pd
import json
from pathlib import Path

from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

In [2]:
from bsm_multi_agents.agents import utils
importlib.reload(utils)
from bsm_multi_agents.agents.utils import get_tool_result_from_messages,print_resp

In [3]:
project_path = '/Users/yifanli/Github/model_doc_automation'

In [4]:
# df_new = pd.read_json(StringIO(json_str), orient='records')

# Graph

In [7]:
from langgraph.graph import StateGraph, END, START

from bsm_multi_agents.graph import state
importlib.reload(state)
from bsm_multi_agents.graph.state import WorkflowState
from bsm_multi_agents.agents.data_loader_agent import data_loader_node

graph = StateGraph(WorkflowState)
graph.add_node("data_loader", data_loader_node)

graph.add_edge(START, "data_loader")
graph.add_edge("data_loader", END)

app = graph.compile()

In [8]:
file_path = os.path.join(project_path, "data/input/dummy_options.csv")
init_state: WorkflowState = {
    "csv_file_path": str(file_path),
    "messages": [HumanMessage(content=f"Load CSV from: {file_path}")],
}

final_state = app.invoke(
    init_state,
    config={"configurable": {"thread_id": "run-1"}}
)

[data_loader] LLM: ChatOllama (qwen2.5:7b), Tools: ['csv_loader']


# Data Loader

## Function Level

In [ ]:
file_path = os.path.join(project_path, "data/input/dummy_options.csv")
df = pd.read_csv(file_path)
json_str = df.to_json(orient='records')

In [5]:
json.dumps({"state_update": {"csv_data": json_str}})

'{"state_update": {"csv_data": "[{\\"date\\":\\"2025-09-01\\",\\"S\\":100,\\"K\\":105,\\"T\\":1.0,\\"r\\":0.05,\\"sigma\\":0.2,\\"option_type\\":\\"call\\"},{\\"date\\":\\"2025-09-02\\",\\"S\\":102,\\"K\\":106,\\"T\\":0.9,\\"r\\":0.045,\\"sigma\\":0.19,\\"option_type\\":\\"put\\"},{\\"date\\":\\"2025-09-03\\",\\"S\\":98,\\"K\\":104,\\"T\\":0.8,\\"r\\":0.048,\\"sigma\\":0.21,\\"option_type\\":\\"call\\"},{\\"date\\":\\"2025-09-04\\",\\"S\\":101,\\"K\\":107,\\"T\\":0.7,\\"r\\":0.047,\\"sigma\\":0.18,\\"option_type\\":\\"call\\"},{\\"date\\":\\"2025-09-05\\",\\"S\\":99,\\"K\\":103,\\"T\\":0.6,\\"r\\":0.046,\\"sigma\\":0.22,\\"option_type\\":\\"put\\"},{\\"date\\":\\"2025-09-06\\",\\"S\\":103,\\"K\\":108,\\"T\\":0.5,\\"r\\":0.049,\\"sigma\\":0.2,\\"option_type\\":\\"put\\"},{\\"date\\":\\"2025-09-07\\",\\"S\\":97,\\"K\\":102,\\"T\\":0.4,\\"r\\":0.044,\\"sigma\\":0.23,\\"option_type\\":\\"call\\"},{\\"date\\":\\"2025-09-08\\",\\"S\\":100,\\"K\\":106,\\"T\\":0.3,\\"r\\":0.05,\\"sigma\\":0.19

## Tool Level

In [6]:
from bsm_multi_agents.tools import data_loader_tools
importlib.reload(data_loader_tools)
from bsm_multi_agents.tools.data_loader_tools import csv_loader

In [ ]:
file_path = os.path.join(project_path, "data/input/dummy_options.csv")
csv_loader.invoke(file_path)

'[{"date":"2025-09-01","S":100,"K":105,"T":1.0,"r":0.05,"sigma":0.2,"option_type":"call"},{"date":"2025-09-02","S":102,"K":106,"T":0.9,"r":0.045,"sigma":0.19,"option_type":"put"},{"date":"2025-09-03","S":98,"K":104,"T":0.8,"r":0.048,"sigma":0.21,"option_type":"call"},{"date":"2025-09-04","S":101,"K":107,"T":0.7,"r":0.047,"sigma":0.18,"option_type":"call"},{"date":"2025-09-05","S":99,"K":103,"T":0.6,"r":0.046,"sigma":0.22,"option_type":"put"},{"date":"2025-09-06","S":103,"K":108,"T":0.5,"r":0.049,"sigma":0.2,"option_type":"put"},{"date":"2025-09-07","S":97,"K":102,"T":0.4,"r":0.044,"sigma":0.23,"option_type":"call"},{"date":"2025-09-08","S":100,"K":106,"T":0.3,"r":0.05,"sigma":0.19,"option_type":"call"},{"date":"2025-09-09","S":104,"K":109,"T":0.2,"r":0.045,"sigma":0.21,"option_type":"put"},{"date":"2025-09-10","S":96,"K":101,"T":0.1,"r":0.048,"sigma":0.2,"option_type":"put"}]'

## Agent Level

In [8]:
from bsm_multi_agents.agents.agent_factory import built_graph_agent_by_role
from bsm_multi_agents.graph.state import WorkflowState
from bsm_multi_agents.prompts.loader import load_prompt

In [ ]:
file_path = os.path.join(project_path, "data/input/dummy_options.csv")
state = {
    "csv_file_path": file_path
}
prompt_path = os.path.join(project_path, "src/bsm_multi_agents/prompts/data_loader_prompts.txt")

In [10]:
agent_role = 'data_loader'
agent = built_graph_agent_by_role(agent_role)

[data_loader] LLM: ChatOllama (qwen2.5:7b), Tools: ['csv_loader']


In [11]:
csv_path = state.get("csv_file_path")
prompt = load_prompt(prompt_path).format(csv_path=str(csv_path))
msg = HumanMessage(content=prompt)

result = agent.invoke(
        {"messages": [msg]},
        config={
            "recursion_limit": 10,
            "configurable": {"thread_id": "run-1"}
        }
    )

In [12]:
csv_data = get_tool_result_from_messages(result["messages"], "csv_loader")

In [13]:
csv_data

{'result': '[{"date":"2025-09-01","S":100,"K":105,"T":1.0,"r":0.05,"sigma":0.2,"option_type":"call"},{"date":"2025-09-02","S":102,"K":106,"T":0.9,"r":0.045,"sigma":0.19,"option_type":"put"},{"date":"2025-09-03","S":98,"K":104,"T":0.8,"r":0.048,"sigma":0.21,"option_type":"call"},{"date":"2025-09-04","S":101,"K":107,"T":0.7,"r":0.047,"sigma":0.18,"option_type":"call"},{"date":"2025-09-05","S":99,"K":103,"T":0.6,"r":0.046,"sigma":0.22,"option_type":"put"},{"date":"2025-09-06","S":103,"K":108,"T":0.5,"r":0.049,"sigma":0.2,"option_type":"put"},{"date":"2025-09-07","S":97,"K":102,"T":0.4,"r":0.044,"sigma":0.23,"option_type":"call"},{"date":"2025-09-08","S":100,"K":106,"T":0.3,"r":0.05,"sigma":0.19,"option_type":"call"},{"date":"2025-09-09","S":104,"K":109,"T":0.2,"r":0.045,"sigma":0.21,"option_type":"put"},{"date":"2025-09-10","S":96,"K":101,"T":0.1,"r":0.048,"sigma":0.2,"option_type":"put"}]'}

### main level

In [14]:
from bsm_multi_agents.agents.data_loader_agent import data_loader_node

In [ ]:
file_path = os.path.join(project_path, "data/input/dummy_options.csv")
state = WorkflowState(csv_file_path=file_path)
out = data_loader_node(state)
out

[data_loader] LLM: ChatOllama (qwen2.5:7b), Tools: ['csv_loader']


{'messages': [HumanMessage(content='Load the option data from the CSV file at: /Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv\n\nUse the csv_loader tool to read the CSV file. Return the data in JSON format.', additional_kwargs={}, response_metadata={}, id='267885a8-cea2-4fd4-9097-85a9702523e4'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-11-25T16:43:01.059636Z', 'done': True, 'done_reason': 'stop', 'total_duration': 532487583, 'load_duration': 61030542, 'prompt_eval_count': 256, 'prompt_eval_duration': 61568667, 'eval_count': 36, 'eval_duration': 397366084, 'model_name': 'qwen2.5:7b', 'model_provider': 'ollama'}, id='lc_run--0f63b34c-e790-41bb-a0a3-29ddc0e8d576-0', tool_calls=[{'name': 'csv_loader', 'args': {'filepath': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv'}, 'id': 'd2a956ac-a5c2-4180-b00e-8c2403d46d17', 'type': 'tool_call'}], usage_metadata={'input_tokens': 256